In [7]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow import keras
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedShuffleSplit
import time
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.linear_model import SGDRegressor

In [9]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP4_Distance (mm)',
                    'AP1_StdDev (mm)', 'AP4_StdDev (mm)',
                    'AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi']  

scaler_columns = ['AP1_Distance (mm)', 'AP4_Distance (mm)', 
                  'AP1_StdDev (mm)', 'AP4_StdDev (mm)', 
                  'AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi',
                  'AP2_Distance_predicted', 'AP3_Distance_predicted']

label_column = 'Label'
target_column = 'Label'


#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [ ]:
ap = 'AP1&AP4'
root = 'AP1 AP4 best (MDE_0.0107Accuracy_0.9917 )'

dataamount = 80
N_val = 16

N_train = dataamount # 訓練集每個類別至少要有 N_train 筆資料
test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

weekrepresent = [['0','0'],['0','1'],['1','2'],['2','3'],['3','4'],['4','5']]
alldatadate = ['2024_12_21','2024_12_27','2025_01_03','2025_01_10','2025_02_28']
number_of_week = len(alldatadate)

ALL_trainingtime = []
ALL_loss = []
ALL_accuracy = []
ALL_mean_mde = []



In [11]:
for i in range(number_of_week):

    best_mde = float('inf')  # 初始化最佳 MDE
    best_trainingtime = None
    best_loss = None
    best_accuracy = None

    for tofindbest in range(3):
        # 載入 regressor
        
        base_model_reg = joblib.load(f'{root}/regressor_model_{ap}_best_{weekrepresent[i][0]}week_to_{weekrepresent[i][1]}week_{dataamount}dataPerRP.pkl')
        # 載入 regressor dnn
        base_model_dnn = load_model(f'{root}/regressor_dnn_{ap}_best_{weekrepresent[i][0]}week_to_{weekrepresent[i][1]}week_{dataamount}dataPerRP.h5')
        # 載入 scaler
        scaler = joblib.load(f'{root}/scaler_regressor_dnn_{ap}_best_0.pkl')

        # 讀取測試資料 2024_12_21   2024_12_27   2025_01_03   2025_01_10   2025_02_28
        test_file_path = f"timestamp_allignment_Balanced_{alldatadate[i]}_rtt_logs.csv"  # 測試資料的檔案名稱
        date_test = f"{alldatadate[i]}"
        modelname = f"regressor_DNN {ap}s BEST_{dataamount}data_{alldatadate[i]}"
        data = pd.read_csv(test_file_path, usecols=selected_columns)


        # DNN transfer learnging 凍結所有層
        for layer in base_model_dnn.layers[:-1]:  # 除了最後一層 (Output Layer)
            layer.trainable = False

        # 確認哪些層可訓練
        base_model_dnn.summary()

        base_model_dnn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])


        data_imputed = data.groupby(label_column).apply(
            lambda group: group.fillna(group.mean())
        ).reset_index()

        print("original data columns :")
        print(data_imputed.columns)


        reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

        y = data_imputed[target_column]

        y_numeric = y.map(reverse_label_mapping)
        y_numeric

        # 把label部分拿掉
        X = data_imputed.drop(columns=['level_1','Label'])

        # 轉為 DataFrame 方便操作
        data = pd.DataFrame(X)
        data['label'] = y_numeric  # 加入 label 欄位

        train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
        train_data_full

        if N_val > 0:
            sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / N_train) # , random_state=42
            train_index, val_index = next(sss.split(train_data_full.drop(columns=['label']), train_data_full['label']))
            train_data = train_data_full.iloc[train_index]
            val_data = train_data_full.iloc[val_index]
            
        else:
            val_data = pd.DataFrame(columns=data.columns)  # 若沒有 validation data，建立空 DataFrame\
            train_data = train_data_full

        # 剩下的資料（未被抽入 train_data_full 的部分）直接作為 test set
        remaining_data = data.drop(train_data_full.index)
        train_data

        # 訓練 regressor 並儲存
        ap1_data = train_data[['AP1_Rssi', 'AP1_Distance (mm)']].dropna().rename(
            columns={'AP1_Rssi': 'Rssi', 'AP1_Distance (mm)': 'Distance'}
        )
        ap4_data = train_data[['AP4_Rssi', 'AP4_Distance (mm)']].dropna().rename(
            columns={'AP4_Rssi': 'Rssi', 'AP4_Distance (mm)': 'Distance'}
        )

        train_data_reg = pd.concat([ap1_data, ap4_data], ignore_index=True)


        X_train_reg = train_data_reg[['Rssi']]
        y_train_reg = train_data_reg['Distance']

        # 重新訓練模型（fine tuning）
        model_reg_finetuned = base_model_reg
        model_reg_finetuned.partial_fit(X_train_reg, y_train_reg)

        # 檢查更新後的係數與截距
        print("更新後模型的係數：", model_reg_finetuned.coef_, "截距：", model_reg_finetuned.intercept_)

        

        ### train data
        train_data['AP2_Distance_predicted'] = np.nan
        train_data['AP3_Distance_predicted'] = np.nan

        # 利用 AP2_Rssi 預測 AP2_Distance_predicted
        mask_ap2 = train_data['AP2_Rssi'].notna()
        train_data.loc[mask_ap2, 'AP2_Distance_predicted'] = model_reg_finetuned.predict(
            train_data.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'})
        )

        # 利用 AP3_Rssi 預測 AP3_Distance_predicted
        mask_ap3 = train_data['AP3_Rssi'].notna()
        train_data.loc[mask_ap3, 'AP3_Distance_predicted'] = model_reg_finetuned.predict(
            train_data.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'})
        )

        ### val data
        val_data['AP2_Distance_predicted'] = np.nan
        val_data['AP3_Distance_predicted'] = np.nan

        # 利用 AP2_Rssi 預測 AP2_Distance_predicted
        mask_ap2 = val_data['AP2_Rssi'].notna()
        val_data.loc[mask_ap2, 'AP2_Distance_predicted'] = model_reg_finetuned.predict(
            val_data.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'})
        )

        # 利用 AP3_Rssi 預測 AP3_Distance_predicted
        mask_ap3 = val_data['AP3_Rssi'].notna()
        val_data.loc[mask_ap3, 'AP3_Distance_predicted'] = model_reg_finetuned.predict(
            val_data.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'})
        )

        ### remaining data
        remaining_data['AP2_Distance_predicted'] = np.nan
        remaining_data['AP3_Distance_predicted'] = np.nan

        # 利用 AP2_Rssi 預測 AP2_Distance_predicted
        mask_ap2 = remaining_data['AP2_Rssi'].notna()
        remaining_data.loc[mask_ap2, 'AP2_Distance_predicted'] = model_reg_finetuned.predict(
            remaining_data.loc[mask_ap2, ['AP2_Rssi']].rename(columns={'AP2_Rssi': 'Rssi'})
        )

        # 利用 AP3_Rssi 預測 AP3_Distance_predicted
        mask_ap3 = remaining_data['AP3_Rssi'].notna()
        remaining_data.loc[mask_ap3, 'AP3_Distance_predicted'] = model_reg_finetuned.predict(
            remaining_data.loc[mask_ap3, ['AP3_Rssi']].rename(columns={'AP3_Rssi': 'Rssi'})
        )

        # 更新 DNN 模型用的特徵欄位，將 regressor 預測值加入
        selected_columns_dnn = selected_columns + ['AP2_Distance_predicted', 'AP3_Distance_predicted']

        print("new added feature column: ")
        print(list(train_data.columns))


        # -------------------------------
        # 準備 DNN 模型的輸入
        # 這邊重點：保留 DataFrame 結構，依據 scaler_columns 選取正確欄位，再進行 scaler.transform
        # -------------------------------
        # 選取訓練、驗證、測試集中的 scaler_columns（順序需與訓練時一致）
        X_train_df = train_data[scaler_columns].copy()
        y_train = train_data['label'].values

        X_val_df = val_data[scaler_columns].copy()
        y_val = val_data['label'].values

        X_test_df = remaining_data[scaler_columns].copy()
        y_test = remaining_data['label'].values

        # 使用預先訓練的 scaler 進行標準化
        X_scaled_train = scaler.transform(X_train_df)
        X_scaled_val = scaler.transform(X_val_df)
        X_scaled_test = scaler.transform(X_test_df)

        # print("訓練資料數量：", len(X_scaled_train))
        # print("驗證資料數量：", len(X_scaled_val))
        # print("測試資料數量：", len(X_scaled_test))


        # **轉換為 NumPy 陣列**
        X_train, y_train = train_data.drop(columns=['label']).values, train_data['label'].values
        X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
        X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values


        # **計算每個 Set 內各 Label 的資料數量**
        train_label_counts = pd.Series(y_train).value_counts().sort_index()
        val_label_counts = pd.Series(y_val).value_counts().sort_index()
        test_label_counts = pd.Series(y_test).value_counts().sort_index()

        # **確保所有 Labels 都有出現在三個 Set 裡**
        all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
        label_distribution = pd.DataFrame(index=all_labels)

        label_distribution["Training Set"] = train_label_counts
        label_distribution["Validation Set"] = val_label_counts
        label_distribution["Test Set"] = test_label_counts

        # **用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）**
        label_distribution = label_distribution.fillna(0).astype(int)

        # from IPython.display import display
        # display(label_distribution)

        start_time = time.time()

        if N_val > 0:
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            base_model_dnn.fit(X_scaled_train, y_train,
                            validation_data=(X_scaled_val, y_val),
                            epochs=10000, batch_size=min(32, max(8, len(X_scaled_train) // 2)),
                            callbacks=[early_stop])
        else:
            early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
            base_model_dnn.fit(X_scaled_train, y_train,
                            epochs=10000, batch_size=min(32, max(8, len(X_scaled_train) // 2)),
                            callbacks=[early_stop])

        end_time = time.time()

        training_time = end_time - start_time
        # print(f"訓練時間：{training_time:.2f} 秒")
        

        # Accuracy
        loss, accuracy = base_model_dnn.evaluate(X_scaled_test, y_test)
        # print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


    # MDE
        # 預測測試資料
        y_test_pred_numeric = base_model_dnn.predict(X_scaled_test)
        y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

        # 轉換為原本的 Label
        y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1

        # 讀取測試資料的實際 Label
        y_test_actual = [label_mapping[str(num + 1)] for num in y_test]  # 補回 +1

        # 取得預測與實際座標
        y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
        y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

        # 計算 MDE (Mean Distance Error)
        distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
        mean_mde = np.mean(distances)

        # 記錄每個 RP 的 MDE
        mde_report_test = {}
        for true_label, distance in zip(y_test_actual, distances):
            if true_label not in mde_report_test:
                mde_report_test[true_label] = []
            mde_report_test[true_label].append(distance)

        # 計算測試資料的 MDE 平均值
        mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}


        # 有更新 
        if mean_mde < best_mde:
            # 儲存 MDE 結果到 JSON 檔案
            test_file_path = f"{root}/{modelname}.json"
            with open(test_file_path, "w") as f:
                json.dump(mde_report_test_avg, f, indent=4)

            print(f"Test Data MDE report saved to: {test_file_path}")
            print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")

            best_mde = mean_mde
            best_trainingtime = training_time
            best_loss = loss
            best_accuracy = accuracy

            # 儲存更新後的regressor and dnn model
            joblib.dump(model_reg_finetuned,f'{root}/regressor_model_{ap}_best_{weekrepresent[i+1][0]}week_to_{weekrepresent[i+1][1]}week_{dataamount}dataPerRP.pkl')
            base_model_dnn.save(f'{root}/regressor_dnn_{ap}_best_{weekrepresent[i+1][0]}week_to_{weekrepresent[i+1][1]}week_{dataamount}dataPerRP.h5')
            print(f"base_model_dnn saved to: {f'{root}/regressor_dnn_{ap}_best_{weekrepresent[i+1][0]}week_to_{weekrepresent[i+1][1]}week_{dataamount}dataPerRP.h5'}")
    
    # 內層迴圈結束後，只存這一次的最佳結果
    ALL_trainingtime.append(best_trainingtime)
    ALL_mean_mde.append(best_mde)
    ALL_loss.append(best_loss)
    ALL_accuracy.append(best_accuracy)



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 49)             │         3,185 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,467 (79.95 KB)

 Trainable params: 3,185 (12.44 KB)

 Non-trainable params: 17,280 (67.50 KB)

...

 Non-trainable params: 17,280 (67.50 KB)

 Optimizer params: 2 (12.00 B)

original data columns :
Index(['Label', 'level_1', 'AP1_Distance (mm)', 'AP4_Distance (mm)',
       'AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi', 'AP1_StdDev (mm)',
       'AP4_StdDev (mm)'],
      dtype='object')
更新後模型的係數： [-276.3796742] 截距： [-8323.25106372]
new added feature column: 
['AP1_Distance (mm)', 'AP4_Distance (mm)', 'AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi', 'AP1_StdDev (mm)', 'AP4_StdDev (mm)', 'label', 'AP2_Distance_predicted', 'AP3_Distance_predicted']
Epoch 1/10000


C:\Users\吳定洋\AppData\Local\Temp\ipykernel_19684\244354728.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['AP2_Distance_predicted'] = np.nan
C:\Users\吳定洋\AppData\Local\Temp\ipykernel_19684\244354728.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['AP3_Distance_predicted'] = np.nan
C:\Users\吳定洋\AppData\Local\Temp\ipykernel_19684\244354728.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2405 - loss: 9.0277 - val_accuracy: 0.3469 - val_loss: 5.9390
Epoch 2/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3595 - loss: 5.5215 - val_accuracy: 0.4056 - val_loss: 4.6848
Epoch 3/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3929 - loss: 4.8177 - val_accuracy: 0.4388 - val_loss: 3.7264
Epoch 4/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4365 - loss: 3.5411 - val_accuracy: 0.4643 - val_loss: 3.0058
Epoch 5/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4558 - loss: 3.1939 - val_accuracy: 0.4821 - val_loss: 2.3236
Epoch 6/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4947 - loss: 2.4614 - val_accuracy: 0.5204 - val_loss: 1.8378
Epoch 7/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5515 - loss: 2.0845 - val_accuracy: 0.6173 - val_loss: 1.4634
Epoch 8/10000
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6424 - loss: 1.5564 - val_accuracy: 0.7066 -

In [12]:
print("training time")
print("[",end = '')
for t in ALL_trainingtime:
    print(f"{t:.2f}",end = ', ')
print("]")

print("Loss")
print("[",end = '')
for l in ALL_loss:
    print(f"{l:.4f}",end = ', ')
print("]")


print("Accuracy")
print("[",end = '')
for a in ALL_accuracy:
    print(f"{a:.4f}",end = ', ')
print("]")


print("MDE")
print("[",end = '')
for m in ALL_mean_mde:
    print(f"{m:.4f}",end = ', ')
print("]")

training time
[13.61, 13.88, 11.08, 11.49, 8.80, ]
Loss
[0.1643, 0.1398, 0.1350, 0.1492, 0.1520, ]
Accuracy
[0.9673, 0.9755, 0.9743, 0.9673, 0.9730, ]
MDE
[0.0554, 0.0413, 0.0392, 0.0433, 0.0408, ]
